Initialise code for google colab

Mount google drive

In [3]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)


Mounted at /content/drive


In [2]:
import tensorflow as tf
print(tf.__version__)
import tensorflow_addons as tfa
print(tfa.__version__)

2.11.0
0.19.0


In [1]:
!pip install tensorflow==2.11
!pip install tensorflow-addons==0.19

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 588.3/588.3 MB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 62.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 439.2/439.2 KB 18.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 29.5 MB/s eta 0:00:00
  Attempting uninstall: flatbuffers
    Found existing installation: flatbuffers 1.12
    Uninstalling flatbuffers-1.12:
      Successfully uninstalled flatbuffers-1.12
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.9.0
    Uninstalling tensorflow-estimator-2.9.0:
      Successfully uninstalled tensorflow-estimator-2.9.0
  Attempting uninstall: keras
    Found existing installation: keras 2.9.0
    Uninstalling keras-2.9.0:
      Successfully uninstalled keras-2.9.0
  Attempting uninstall: tensorboard
    Found existing i

Create data base files under google colab environment

In [4]:
!unzip -q '/content/drive/MyDrive/data_equalize.zip' -d '/content/'

Define working directory to our jupyter repertory:
* because path to the different repertories (./data, ./output...) are define relatevly to jupyter one
* let import _mypath which add ./lib to python path in order to import our own define libraries


In [6]:
# for google colab use
%cd /content/drive/MyDrive/covid-19-xRay/jupyter
from google.colab.patches import cv2_imshow
db_work_dir = '/content'


/content/drive/MyDrive/covid-19-xRay/jupyter


In [1]:
# for local use
db_work_dir = '..'

In [2]:
import _mypath
import os
import shutil
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import cv2
from PIL import Image
import joblib

%load_ext autoreload
%autoreload 1

2023-01-30 09:33:38.307799: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
%aimport database.path_origin_data
%aimport database.dataset

from database.path_origin_data import build_data_paths 
from database.path_origin_data import lung_name, infection_name
from database.path_origin_data import train_name, test_name, valid_name
from database.path_origin_data import normal_name, covid_name, no_covid_name
from database.path_origin_data import images_name, lung_mask_name, infection_mask_name

from database.dataset import build_dataset

%aimport run_exp.test
%aimport run_exp.classif_autoencoder
%aimport run_exp.standard

from run_exp.test import compile_test_model, test_accuracy, test_conf_mat
from run_exp.classif_autoencoder import run_experiment as run_experiment_autoencoder
from run_exp.standard import run_experiment as run_experiment_pure_cnn



Build paths and variables for reading data base hierarchy

In [4]:
# input
db_name = 'data_equalize'
db_path = os.path.join(db_work_dir, db_name)

# output
output_path = os.path.join('..', 'output', 'cnn_transformer')
if not os.path.exists(output_path):
    os.makedirs(output_path, exist_ok=True)

log_path = os.path.join(output_path, 'log')
if not os.path.exists(log_path):
    os.makedirs(log_path, exist_ok=True)

ckpt_path = os.path.join(output_path, 'ckpt')
if not os.path.exists(ckpt_path):
    os.makedirs(ckpt_path, exist_ok=True)

metric_path = os.path.join(output_path, 'metric')
if not os.path.exists(metric_path):
    os.makedirs(metric_path, exist_ok=True)


activation_path = os.path.join(output_path, 'activation')
if not os.path.exists(activation_path):
    os.makedirs(activation_path, exist_ok=True)

pattern_path = os.path.join(output_path, 'pattern')
if not os.path.exists(pattern_path):
    os.makedirs(pattern_path, exist_ok=True)

occultation_path = os.path.join(output_path, 'occulatation')
if not os.path.exists(occultation_path):
    os.makedirs(occultation_path, exist_ok=True)

grad_cam_path = os.path.join(output_path, 'grad_cam')
if not os.path.exists(grad_cam_path):
    os.makedirs(grad_cam_path, exist_ok=True)


Structure to manage paths in data base

In [5]:
data_paths = build_data_paths()
idx = pd.IndexSlice

Create tf Dataset

In [12]:

paths = data_paths['path']

ds_train, ds_train_file_paths = build_dataset(db_path, paths, db=[lung_name], ds=[train_name])
ds_test, ds_test_file_paths = build_dataset(db_path, paths, db=[lung_name], ds=[test_name])
ds_valid, ds_valid_file_paths = build_dataset(db_path, paths, db=[lung_name], ds=[valid_name])
print(ds_train.cardinality().numpy())
print(len(ds_train_file_paths))


Found 6849 files belonging to 1 classes.
Found 7658 files belonging to 1 classes.
Found 7208 files belonging to 1 classes.
Found 2140 files belonging to 1 classes.
Found 2394 files belonging to 1 classes.
Found 2253 files belonging to 1 classes.
Found 1712 files belonging to 1 classes.
Found 1902 files belonging to 1 classes.
Found 1802 files belonging to 1 classes.
21715
21715


Model parameters

In [12]:
batch_size_cnn = 32
batch_size_transformer = 32

image_size = 256
input_shape = (image_size, image_size, 1)

learning_rate = 0.001
weight_decay = 0.0001
num_epochs = 100

label_smoothing = 0.1
lam_recon = 10.
patience = 5
min_delta = 0.005
min_delta_fine_tuning = 0.0005

# data augmentation
scale = 1. / 255.
flip = "horizontal"
rotation_factor = 10. / 360.
zoom_height_factor = 0.2
zoom_width_factor = 0.2

# vit
patch_size = 1
transformer_layers = 4
num_heads = 8
projection_dim = 64
transformer_units_rate = [2, 1]
mlp_head_units = [1024, 256]  # Size of the dense layers of the final classifier


Data augmentation

In [13]:
augmentation = keras.Sequential(
    [
      layers.Rescaling(scale=scale),
      layers.RandomFlip(flip),
      layers.RandomRotation(rotation_factor),
      layers.RandomZoom(height_factor=zoom_height_factor, width_factor=zoom_width_factor),
    ],
    name='augmentation'
)

cnn encoder

In [14]:
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Dropout

encoder = keras.Sequential(
    [
      layers.Conv2D(128, (3, 3), activation = 'relu', padding='same', kernel_initializer='random_normal'),
      layers.MaxPooling2D(pool_size = (2, 2)),
      layers.Conv2D(128, (3, 3), activation = 'relu', padding='same', kernel_initializer='random_normal'),
      layers.MaxPooling2D(pool_size = (2, 2)),
      layers.Conv2D(64, (3, 3), activation = 'relu', padding='same', kernel_initializer='random_normal'),
      layers.MaxPooling2D(pool_size = (2, 2)),
      layers.Conv2D(64, (3, 3), activation = 'relu', padding='same', kernel_initializer='random_normal'),
      layers.MaxPooling2D(pool_size = (2, 2)),
      layers.Conv2D(64, (3, 3), activation = 'relu', padding='same', kernel_initializer='random_normal'),
      layers.MaxPooling2D(pool_size = (2, 2)),
      layers.Flatten(),
    ],
    name='encoder'
)

decoder

In [ ]:
decoder = keras.Sequential(
    [
      layers.Dense(1024, activation='relu'),
      layers.Dense(256, activation='relu'),
      layers.Dense(np.prod(input_shape), activation='sigmoid'),
      layers.Reshape(target_shape=input_shape),
    ],
    name='decoder'
)


In [ ]:
classifier = keras.Sequential(
    [
      layers.Dense(1024, activation='relu'),
      layers.Dense(256, activation='relu'),
      layers.Dense(3, activation='softmax'),
    ],
    name='classifier'
)


In [ ]:
class MyModel(tf.keras.Model):

  def __init__(self, *args, **kwargs):
    super().__init__(*args, **kwargs)
    self.augmentation = augmentation
    self.encoder = encoder
    self.decoder = decoder
    self.classifier = classifier

  def call(self, inputs):
    x = self.augmentation(inputs)
    x = self.encoder(x)
    self.decoder(x)
    return self.classifier(x)

classif_decoder = MyModel(name='classif_decoder')

inputs = layers.Input(shape=input_shape)
classified_decoded = classif_decoder(inputs)

model = keras.Model(inputs=inputs, outputs=classified_decoded)
model.get_layer('classif_decoder').summary()



Model: "classif_decoder"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 augmentation (Sequential)   (None, 256, 256, 1)       0         
                                                                 
 encoder (Sequential)        (None, 4096)              296512    
                                                                 
 decoder (Sequential)        (None, 256, 256, 1)       21300480  
                                                                 
 classifier (Sequential)     (None, 3)                 4458499   
                                                                 
Total params: 26,055,491
Trainable params: 26,055,491
Non-trainable params: 0
_________________________________________________________________


In [ ]:
classif_decoder = model.get_layer('classif_decoder')
encoder_input = classif_decoder.get_layer('augmentation')
decoder_output = classif_decoder.get_layer('decoder')

model_name = 'cnn'
cnn_history, cnn_conf_mat = run_experiment_autoencoder(
    model, encoder_input, decoder_output,
    ds_train, ds_valid, ds_test,
    batch_size=batch_size_cnn, num_epochs=num_epochs,
    learning_rate=learning_rate, weight_decay=weight_decay,
    lam_recon=lam_recon,
    from_logits=False, label_smoothing=label_smoothing,
    patience=patience, min_delta=min_delta,
    log_path=log_path, ckpt_path=ckpt_path,
    prefix=model_name
)


In [ ]:
model_name = 'cnn'
checkpoint_filename = os.path.join(ckpt_path, model_name + 'weights.hdf5')
model.load_weights(checkpoint_filename)


In [ ]:
y_test_pd, y_pred_pd, accuracy, conf_mat, report = compile_test_model(
    model,
    ds_test, batch_size_cnn,
    from_logits=False, label_smoothing=label_smoothing
)

print(f"Test accuracy: {round(accuracy * 100, 2)}%")

print(report)
f_name = os.path.join(metric_path, model_name + '_report.txt')
with open(f_name, "w") as text_file:
  text_file.write(report)

conf_mat
f_name = os.path.join(metric_path, model_name + '_conf_mat.joblib')
joblib.dump(conf_mat, f_name)


213/213 [==============================] - 16s 53ms/step - loss: 0.4266 - accuracy: 0.9347
Test accuracy: 93.47%
213/213 [==============================] - 11s 49ms/step


Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


Predicted,0,1,2
Real,,,
0,1975,32,133
1,40,2298,56
2,136,46,2071


Fine tuning cnn without decoder part

In [ ]:
inputs = layers.Input(shape=input_shape)
augmented = augmentation(inputs)
encoded = encoder(augmented)
classified = classifier(encoded)


In [ ]:
# Create the Keras model.
model_pure_cnn = keras.Model(inputs=inputs, outputs=classified)
model_pure_cnn.summary()


Model: "model_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, 256, 256, 1)]     0         
                                                                 
 augmentation (Sequential)   (None, 256, 256, 1)       0         
                                                                 
 encoder (Sequential)        (None, 4096)              296512    
                                                                 
 classifier (Sequential)     (None, 3)                 4458499   
                                                                 
Total params: 4,755,011
Trainable params: 4,458,499
Non-trainable params: 296,512
_________________________________________________________________


In [ ]:
model_name = 'pure_cnn_lr0.001'
pure_cnn_history, pure_cnn_conf_mat = run_experiment_pure_cnn(
    model_pure_cnn,
    ds_train, ds_valid, ds_test,
    batch_size=batch_size_cnn, num_epochs=num_epochs,
    learning_rate=learning_rate / 1000., weight_decay=weight_decay,
    from_logits=False, label_smoothing=label_smoothing,
    patience=patience, min_delta=min_delta_fine_tuning,
    log_path=log_path, ckpt_path=ckpt_path,
    prefix=model_name
)


Epoch 1/100


679/679 [==============================] - ETA: 0s - loss: 0.4813 - accuracy: 0.8982
Epoch 1: val_accuracy improved from -inf to 0.91876, saving model to ../output/ckpt/pure_cnn_lr0.001_weights.hdf5
679/679 [==============================] - 252s 311ms/step - loss: 0.4813 - accuracy: 0.8982 - val_loss: 0.4737 - val_accuracy: 0.9188
Epoch 2/100
679/679 [==============================] - ETA: 0s - loss: 0.5301 - accuracy: 0.8971
Epoch 2: val_accuracy did not improve from 0.91876
679/679 [==============================] - 225s 302ms/step - loss: 0.5301 - accuracy: 0.8971 - val_loss: 0.5298 - val_accuracy: 0.9165
Epoch 3/100
679/679 [==============================] - ETA: 0s - loss: 0.5960 - accuracy: 0.8878
Epoch 3: val_accuracy did not improve from 0.91876
679/679 [==============================] - 223s 302ms/step - loss: 0.5960 - accuracy: 0.8878 - val_loss: 0.5948 - val_accuracy: 0.9141
Epoch 4/100
679/679 [==============================] - ETA: 0s - loss: 0.6686 - accuracy: 0.8730
Epo

Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


In [ ]:
model_name = 'pure_cnn_lr0.001'
checkpoint_filename = os.path.join(ckpt_path, model_name + '_weights.hdf5')
model_pure_cnn.load_weights(checkpoint_filename)


In [ ]:
y_test_pd, y_pred_pd, accuracy, conf_mat, report = compile_test_model(
    model_pure_cnn,
    ds_test, batch_size_cnn,
    from_logits=False, label_smoothing=label_smoothing
)

print(f"Test accuracy: {round(accuracy * 100, 2)}%")

print(report)
f_name = os.path.join(metric_path, model_name + '_report.txt')
with open(f_name, "w") as text_file:
  text_file.write(report)

conf_mat
f_name = os.path.join(metric_path, model_name + '_conf_mat.joblib')
joblib.dump(conf_mat, f_name)


213/213 [==============================] - 12s 51ms/step - loss: 0.4612 - accuracy: 0.9331
Test accuracy: 93.31%
213/213 [==============================] - 12s 50ms/step


Predicted,0,1,2
Real,,,
0,1988,21,131
1,59,2259,76
2,129,38,2086


Branch transformer on top of cnn.

In [ ]:
encoder.summary()

Model: "encoder"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 256, 256, 128)     1280      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 128, 128, 128)    0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 128, 128, 128)     147584    
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 64, 64, 128)      0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 64, 64, 64)        73792     
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 32, 32, 64)       0   

In [15]:
# Sub Model
shared_encoder = keras.Sequential(name='shared_encoder')

for layer in encoder.layers[:-1]:
  shared_encoder.add(layer)

for layer in shared_encoder.layers:
  layer.trainable = False


In [16]:

%aimport myLayers.vision_transformer
%aimport myLayers.mlp
from myLayers.vision_transformer import add_vit
from myLayers.mlp import mlp

inputs = layers.Input(shape=input_shape)
augmented_transformer = augmentation(inputs)
shared_encoded = shared_encoder(augmented_transformer)
features = add_vit(shared_encoded,
            patch_size=patch_size,
            input_image_size=shared_encoded.shape[1],
            transformer_layers=transformer_layers,
            num_heads=num_heads,
            projection_dim=projection_dim,
            transformer_units_rate=transformer_units_rate,
            mlp_head_units=mlp_head_units)
# Classify outputs.
softmax = layers.Dense(3, activation='softmax', kernel_initializer='random_normal')(features)

# Create the Keras model
model_transformer = keras.Model(inputs=inputs, outputs=softmax)
shared_encoder.summary()
model_transformer.summary()


Model: "shared_encoder"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 256, 256, 128)     1280      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 128, 128, 128)    0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 128, 128, 128)     147584    
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 64, 64, 128)      0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 64, 64, 64)        73792     
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 32, 32, 64)    

In [ ]:
%aimport run_exp.standard
from run_exp.standard import run_experiment as run_experiment_transformer

model_name = 'transformer_8_8_lr0.5'
transformer_history, transformer_conf_mat = run_experiment_transformer(
    model_transformer,
    ds_train, ds_valid, ds_test,
    batch_size=batch_size_transformer, num_epochs=num_epochs,
    learning_rate=learning_rate / 2., weight_decay=weight_decay,
    from_logits=False, label_smoothing=label_smoothing,
    patience=patience, min_delta=min_delta,
    log_path=log_path, ckpt_path=ckpt_path,
    prefix=model_name
)


Epoch 1/100


679/679 [==============================] - ETA: 0s - loss: 0.9223 - accuracy: 0.6175
Epoch 1: val_accuracy improved from -inf to 0.68796, saving model to ../output/ckpt/transformer_8_8_lr0.5_weights.hdf5
679/679 [==============================] - 199s 247ms/step - loss: 0.9223 - accuracy: 0.6175 - val_loss: 0.8249 - val_accuracy: 0.6880
Epoch 2/100
679/679 [==============================] - ETA: 0s - loss: 0.6964 - accuracy: 0.7835
Epoch 2: val_accuracy improved from 0.68796 to 0.71104, saving model to ../output/ckpt/transformer_8_8_lr0.5_weights.hdf5
679/679 [==============================] - 182s 244ms/step - loss: 0.6964 - accuracy: 0.7835 - val_loss: 0.8053 - val_accuracy: 0.7110
Epoch 3/100
679/679 [==============================] - ETA: 0s - loss: 0.6559 - accuracy: 0.8029
Epoch 3: val_accuracy improved from 0.71104 to 0.71806, saving model to ../output/ckpt/transformer_8_8_lr0.5_weights.hdf5
679/679 [==============================] - 183s 246ms/step - loss: 0.6559 - accuracy: 0.

Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


In [ ]:
model_name = 'transformer_8_8_lr0.5_all_layers_lr'
checkpoint_filename = os.path.join(ckpt_path, model_name + '_weights.hdf5')
model_transformer.load_weights(checkpoint_filename)


In [ ]:
y_test_pd, y_pred_pd, accuracy, conf_mat, report = compile_test_model(
    model_transformer,
    ds_test, batch_size_transformer,
    from_logits=False, label_smoothing=label_smoothing
)

print(f"Test accuracy: {round(accuracy * 100, 2)}%")

print(report)
f_name = os.path.join(metric_path, model_name + '_report.txt')
with open(f_name, "w") as text_file:
  text_file.write(report)

conf_mat
f_name = os.path.join(metric_path, model_name + '_conf_mat.joblib')
joblib.dump(conf_mat, f_name)


213/213 [==============================] - 26s 72ms/step - loss: 0.4304 - accuracy: 0.9262
Test accuracy: 92.62%
213/213 [==============================] - 14s 59ms/step


Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


Predicted,0,1,2
Real,,,
0,2030,13,97
1,69,2234,91
2,213,18,2022


In [ ]:
for layer in shared_encoder.layers:
  layer.trainable = True

In [ ]:
%aimport run_exp.standard
from run_exp.standard import run_experiment as run_experiment_transformer

model_name = 'transformer_8_8_lr0.5_all_layers_lr_fine_tuning_lr0.0001'
transformer_fine_tuning_history, transformer_fine_tuning_conf_mat = run_experiment_transformer(
    model_transformer,
    ds_train, ds_valid, ds_test,
    batch_size=batch_size_transformer, num_epochs=num_epochs,
    learning_rate=learning_rate / 10000., weight_decay=weight_decay,
    from_logits=False, label_smoothing=label_smoothing,
    patience=patience, min_delta=min_delta_fine_tuning,
    log_path=log_path, ckpt_path=ckpt_path,
    prefix=model_name
)


In [17]:
model_name = 'transformer_8_8_lr0.5_all_layers_lr_fine_tuning_lr0.0001'
checkpoint_filename = os.path.join(ckpt_path, model_name + '_weights.hdf5')
model_transformer.load_weights(checkpoint_filename)


In [ ]:
# test_accuracy(
#     model_transformer,
#     ds_train, batch_size_transformer,
# )
# test_accuracy(
#     model_transformer,
#     ds_valid, batch_size_transformer,
# )
# test_accuracy(
#     model_transformer,
#     ds_test, batch_size_transformer,
# )

y_test_pd, y_pred_pd, accuracy, conf_mat, report = compile_test_model(
    model_transformer,
    ds_test, batch_size_transformer,
    from_logits=False, label_smoothing=label_smoothing
)

print(f"Test accuracy: {round(accuracy * 100, 2)}%")

print(report)
f_name = os.path.join(metric_path, model_name + '_report.txt')
with open(f_name, "w") as text_file:
  text_file.write(report)

conf_mat
f_name = os.path.join(metric_path, model_name + '_conf_mat.joblib')
joblib.dump(conf_mat, f_name)


213/213 [==============================] - 24s 63ms/step - loss: 0.4454 - accuracy: 0.9334
Test accuracy: 93.34%
213/213 [==============================] - 14s 59ms/step


Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


              precision    recall  f1-score   support

           0       0.94      0.89      0.92      2140
           1       0.98      0.96      0.97      2394
           2       0.88      0.95      0.91      2253

    accuracy                           0.93      6787
   macro avg       0.93      0.93      0.93      6787
weighted avg       0.93      0.93      0.93      6787



Predicted,0,1,2
Real,,,
0,1907,23,210
1,32,2294,68
2,87,32,2134


In [ ]:
ds_test_file_paths_pd = pd.DataFrame(data=ds_test_file_paths, columns=['file_path'])
ds_results_test = pd.concat([ds_test_file_paths_pd, y_test_pd, y_pred_pd])
ds_results_test.head()